In [8]:
import os
import shutil

# Ordnerpfade
labels_dir = "D:/Datasets/YOLO_Dataset/train/labels"
images_dir = "D:/Datasets/YOLO_Dataset/train/images"
nodule_dir = "D:/Datasets/YOLO_Caps/train/nodule"
non_nodule_dir = "D:/Datasets/YOLO_Caps/train/non-nodule"

# Sicherstellen, dass die Zielordner existieren
os.makedirs(nodule_dir, exist_ok=True)
os.makedirs(non_nodule_dir, exist_ok=True)

# Iteriere durch alle Textdateien im Label-Ordner
for label_file in os.listdir(labels_dir):
    if label_file.endswith(".txt"):  # Nur Textdateien berücksichtigen
        label_path = os.path.join(labels_dir, label_file)
        
        # Überprüfe, ob die Datei leer ist
        with open(label_path, "r") as file:
            content = file.read().strip()
        
        # Image-Datei suchen
        image_name = label_file.replace(".txt", ".jpeg")  # oder .jpg, je nach Bildformat
        image_path = os.path.join(images_dir, image_name)
        
        # Kopiere das Bild in den entsprechenden Ordner
        if os.path.exists(image_path):
            if content:  # Datei hat Inhalt (Nodule)
                shutil.copy(image_path, nodule_dir)
            else:  # Datei ist leer (Non-Nodule)
                shutil.copy(image_path, non_nodule_dir)

print("Kopieren abgeschlossen.")


Kopieren abgeschlossen.


kind	non-nodule	nodule

test	2670		2700

val	    2654		2714

train	21520		21430


In [14]:
import os
import random
from PIL import Image
import shutil

# Pfade zu den Ordnern
nodules_folder = "D:/YOLO_Caps/nodule"
non_nodules_folder = "D:/YOLO_Caps/non-nodule"
labels_folder = "D:/YOLO_Caps/labels"
nodule_output_folder = "D:/YOLO_Caps/cropped_nodule"
non_nodule_output_folder = "D:/YOLO_Caps/cropped_non_nodule"

# Zähler für Non-Nodule-Bilder
non_nodule_counter = 0

# Sicherstellen, dass die Zielordner existieren
os.makedirs(nodule_output_folder, exist_ok=True)
os.makedirs(non_nodule_output_folder, exist_ok=True)

# Funktion zum Croppen eines Bildes auf Basis einer Bounding Box
def crop_and_resize(image, bbox, target_size=128):
    x_center, y_center, width, height = bbox
    # Berechne die maximale Seite
    max_side = max(width, height)

    x_center=x_center*512
    y_center=y_center*512
    max_side=max_side*512

    # Berechne die neue quadratische ROI
    x_min = int(x_center - max_side / 2)
    y_min = int(y_center - max_side / 2)
    x_max = int(x_center + max_side / 2)
    y_max = int(y_center + max_side / 2)

    # Stelle sicher, dass die ROI im Bild bleibt
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(image.width, x_max)
    y_max = min(image.height, y_max)
    
    cropped_image = image.crop((x_min, y_min, x_max, y_max))

    cropped_image = cropped_image.resize((target_size, target_size), Image.LANCZOS)
    # Crop das quadratische Bild
    return cropped_image


# Funktion zum Laden der Bounding Boxes aus einer YOLO-Textdatei
def load_bboxes(label_path):
    bboxes = []
    with open(label_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) >= 5:
                # YOLO Format: class_id x_center y_center width height
                class_id, x_center, y_center, width, height = map(float, parts)
                bboxes.append((x_center, y_center, width, height))
    return bboxes

# Gehe durch alle Nodule-Bilder
for nodule_image_name in os.listdir(nodules_folder):
    if nodule_image_name.endswith(".jpeg"):
        # Lade das Nodule-Bild
        nodule_image_path = os.path.join(nodules_folder, nodule_image_name)
        nodule_image = Image.open(nodule_image_path)

        # Lade die zugehörige Label-Datei
        label_path = os.path.join(labels_folder, nodule_image_name.replace(".jpeg", ".txt"))
        
        if os.path.exists(label_path):
            # Lade die Bounding Boxes für das Bild
            bboxes = load_bboxes(label_path)

            # Durchlaufe jede Bounding Box und bearbeite das Bild
            for i, bbox in enumerate(bboxes):
                # Croppen und Resizen des Nodule-Bildes
                cropped_nodule_image = crop_and_resize(nodule_image, bbox)

                # Speichern des Nodule-Bildes im nodule-Ordner
                nodule_filename = f"n{nodule_image_name}"
                cropped_nodule_image.save(os.path.join(nodule_output_folder, nodule_filename))

                # Wähle ein zufälliges Non-Nodule Bild aus
                random_non_nodule_image_name = random.choice(os.listdir(non_nodules_folder))
                if random_non_nodule_image_name.endswith(".jpeg"):
                    random_non_nodule_image_path = os.path.join(non_nodules_folder, random_non_nodule_image_name)
                    random_non_nodule_image = Image.open(random_non_nodule_image_path)
                    
                    # Croppen und Resizen des zufälligen Non-Nodule-Bildes
                    cropped_non_nodule_image = crop_and_resize(random_non_nodule_image, bbox)

                    # Speichern des Non-Nodule-Bildes im cropped_non_nodule-Ordner
                    non_nodule_filename = f"nn{random_non_nodule_image_name.replace('.jpeg', f'_{non_nodule_counter}.jpeg')}"
                    cropped_non_nodule_image.save(os.path.join(non_nodule_output_folder, non_nodule_filename))
                    non_nodule_counter += 1
